In [1]:
import pandas as pd

In [3]:
results_df = pd.read_csv("coil_anomaly_volatility_new.csv")
results_df.head()

,Coil ID,Shape Target Second Coefficient,Stand 1 Predicted Run Force,Stand 1 Gap Stick Offset,Tension Reel Calculated Tension,Stand 1-2 Total Tension Feedback,Stand 3 - Operator Side Force,Stand 1 Run Gap Setpoint,Stand 1 Gap Bite Offset,S1 Operating Bending Trim,...,Stand 2 DS Roll Force,Stand 3 DS Roll Force,Stand 4 DS Roll Force,Roll Force Hydraulics Pressure Feedback,Stand 1 Roll Force,Stand 1 Roll Force limit (g67 delayed 200 ms + g80),Stand 1 Roll Force Increase Limit (based on predicted run force),Stand 4 OS Bending Shape Trim,Stand 4 DS Bending Shape Trim,Shape Target Second Coefficient.1
0,5399358,0.0,0.0,0.0,2.567237,4.645477,5.745721,0.0,0.0,0.0,...,4.034230,5.012225,3.789731,1.711491,1.466993,7.579462,0.122249,4.278729,4.400978,0.0
1,5399575,0.0,0.0,0.0,6.937799,5.263158,2.073365,0.0,0.0,0.0,...,2.870813,6.539075,1.754386,5.741627,3.907496,3.827751,0.000000,1.355662,6.618820,0.0
2,5400215,0.0,0.0,0.0,12.735327,4.318937,0.442968,0.0,0.0,0.0,...,2.990033,3.322259,4.651163,4.208195,7.087486,3.875969,0.000000,3.433001,3.986711,0.0
3,5400236,0.0,0.0,0.0,10.976948,5.268935,3.183315,0.0,0.0,0.0,...,3.622393,4.939627,0.658617,4.171240,4.281010,1.975851,0.109769,9.110867,0.000000,0.0
4,5400244,0.0,0.0,0.0,12.573443,0.705053,5.757932,0.0,0.0,0.0,...,3.290247,0.822562,1.880141,5.757932,3.995300,7.050529,0.000000,2.115159,2.115159,0.0


In [5]:
# Reshape the data frame to long form (already done)
long_df = results_df.melt(id_vars="Coil ID", var_name="Signal", value_name="Anomaly_Percentage")

# Group by each signal to calculate mean and std per signal
stats = long_df.groupby("Signal")["Anomaly_Percentage"].agg(["mean", "std"]).reset_index()

# Merge these stats back to the long_df
long_df = long_df.merge(stats, on="Signal")

# Calculate signal-wise anomaly boundaries
long_df["Upper_Bound"] = long_df["mean"] + 3 * long_df["std"]

# Filter anomalies per signal
anomalies_over_threshold = long_df[
    (long_df["Anomaly_Percentage"] > long_df["Upper_Bound"])
]

# Optionally, select relevant columns
anomalies_over_threshold = anomalies_over_threshold[["Coil ID", "Signal", "Anomaly_Percentage"]]

In [6]:
anomalies_over_threshold.to_csv(f"anomalies_outside_threshold_volatility_new.csv", index=False)

In [51]:
# Cluster wise

In [6]:
# # Melt with Cluster included as id_vars
# long_df = results_df.melt(id_vars=["Coil ID", "K means Cluster"], var_name="Signal", value_name="Anomaly_Percentage")

# # Group by both Signal and Cluster to get stats
# stats = long_df.groupby(["Signal", "K means Cluster"])["Anomaly_Percentage"].agg(["mean", "std"]).reset_index()

# # Merge stats back
# long_df = long_df.merge(stats, on=["Signal", "K means Cluster"])

# # Calculate upper bound per signal-cluster group
# long_df["Upper_Bound"] = long_df["mean"] + 3 * long_df["std"]

# # Filter anomalies by signal and cluster
# anomalies_over_threshold = long_df[
#     (long_df["Anomaly_Percentage"] > long_df["Upper_Bound"])
# ]

# # Keep relevant columns
# anomalies_over_threshold = anomalies_over_threshold[["Coil ID", "K means Cluster", "Signal", "Anomaly_Percentage"]]

In [7]:
# anomalies_over_threshold.to_csv("Anomaly CSV/Bad Coils/anomalies_outside_threshold_isolationforest_clustering_Data.csv", index=False)